# Configuring an Experiment with UW Tools

<div class="alert alert-warning"><b>Note: </b>This notebook was tested using <code>uwtools</code> version 2.4.2.</div>

This notebook demonstrates an example of how to build an experiment configuration file from multiple smaller files, dereference Jinja 2 expressions in the file, and validate the file to check for potential errors. It is meant to demonstrate these steps as part of a larger experimental setup. 
<!--cell 0-->

In [1]:
from datetime import datetime
from uwtools.api.config import get_yaml_config
from uwtools.api import chgres_cube
from uwtools.api.logging import use_uwtools_logger

use_uwtools_logger()

Given multiple configuration files like the ones below, `uwtools` allows us to combine them into one comprehensive configuration file for an experiment. Suppose we start with a base configuration file like the one below, which is a truncated version of the default config for SRW, using FV3GFS initial conditions (ICs) and the FV3_GFS_v16 physics suite.
<!--cell 2-->

In [2]:
%%bash
cat fixtures/config-exp/base-file.yaml

task_make_ics:
  chgres_cube:
    execution:
      executable: "execdir/chgres_cube"
    namelist:
      update_values:
        config:
          cycle_day: !int "{{ cycle.strftime('%d') }}"
          cycle_hour: !int "{{ cycle.strftime('%H') }}"
          cycle_mon: !int "{{ cycle.strftime('%m') }}"
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: "{{ task_make_ics.chgres_cube.rundir }}"
          external_model: "GFS"
          input_type: "gaussian_nemsio"
          mosaic_file_target_grid: "path/to/example_mosaic.halo.nc"
          tg3_from_soil: false
          tracers:
            - sphum
            - liq_wat
          tracers_input:
            - spfh
            - clwmr
          varmap_file: "{{ user.PARMdir }}/ufs_utils/varmap_tables/GFSphys_var_map.txt"
          vcoord_file_target_grid: "path/to/global_hyblev.165.txt"
    rundir: '{{ workflow.EXPTDIR }}/make_ics'


If, instead, a different physics suite such as FV3_RAP is needed for the desired experiment, changes may be applied from a config that describes the set of changes related to the initial condition source.
<!--cell 4-->

In [3]:
%%bash
cat fixtures/config-exp/fv3-rap-physics.yaml

task_make_ics:
  chgres_cube:
    namelist:
      update_values:
        config:
          varmap_file: "{{ user.PARMdir }}/ufs_utils/varmap_tables/GSDphys_var_map.txt"


There will be some information each user will need to provide. Those can be included in a user config file. Separating these items into their own config file will help guide an easier user experience. 
<!--cell 6-->

In [4]:
%%bash
cat fixtures/config-exp/user.yaml

user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/to/ufs-srweather-app/parm
workflow: 
  EXPTDIR: /path/to/my/output


The `get_yaml_config()` function initially creates a `YAMLConfig` object which contains only the `base-file.yaml` config. This is the base from which the comprehensive config file will be built.
<!--cell 8-->

In [5]:
experiment_config = get_yaml_config('fixtures/config-exp/base-file.yaml')
print(experiment_config)

task_make_ics:
  chgres_cube:
    execution:
      executable: execdir/chgres_cube
    namelist:
      update_values:
        config:
          cycle_day: !int '{{ cycle.strftime(''%d'') }}'
          cycle_hour: !int '{{ cycle.strftime(''%H'') }}'
          cycle_mon: !int '{{ cycle.strftime(''%m'') }}'
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: '{{ task_make_ics.chgres_cube.rundir }}'
          external_model: GFS
          input_type: gaussian_nemsio
          mosaic_file_target_grid: path/to/example_mosaic.halo.nc
          tg3_from_soil: false
          tracers:
          - sphum
          - liq_wat
          tracers_input:
          - spfh
          - clwmr
          varmap_file: '{{ user.PARMdir }}/ufs_utils/varmap_tables/GFSphys_var_map.txt'
          vcoord_file_target_grid: path/to/global_hyblev.165.txt
    rundir: '{{ workflow.EXPTDIR }}/make_ics'


Here the list of additional config file paths is created. Note that if the configs share any keys, the values from later updates will override the earlier ones. For example, `varmap_file:` contained a path ending with `GFSphys_var_map.txt` base config file, but it was changed to a path ending with `GSDphys_var_map.txt` by the FV3 RAP physics config. After a `YAMLConfig` object for each config file is created using `get_yaml_config()`, the `update_from()` method updates the base file.
<!--cell 10-->

In [6]:
config_files = ['fixtures/config-exp/fv3-rap-physics.yaml',
                'fixtures/config-exp/user.yaml'
]
for config_file in config_files:
    config = get_yaml_config(config_file)
    experiment_config.update_from(config)

print(experiment_config)

task_make_ics:
  chgres_cube:
    execution:
      executable: execdir/chgres_cube
    namelist:
      update_values:
        config:
          cycle_day: !int '{{ cycle.strftime(''%d'') }}'
          cycle_hour: !int '{{ cycle.strftime(''%H'') }}'
          cycle_mon: !int '{{ cycle.strftime(''%m'') }}'
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: '{{ task_make_ics.chgres_cube.rundir }}'
          external_model: GFS
          input_type: gaussian_nemsio
          mosaic_file_target_grid: path/to/example_mosaic.halo.nc
          tg3_from_soil: false
          tracers:
          - sphum
          - liq_wat
          tracers_input:
          - spfh
          - clwmr
          varmap_file: '{{ user.PARMdir }}/ufs_utils/varmap_tables/GSDphys_var_map.txt'
          vcoord_file_target_grid: path/to/global_hyblev.165.txt
    rundir: '{{ workflow.EXPTDIR }}/make_ics'
user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/t

After all of the key-value pairs are coalesced into one object, the `dereference()` method renders any Jinja 2 expressions wherever possible. Note that keys like `varmap_file:` and `rundir:` have had their Jinja expressions rendered using the information provided by `user.PARMdir` and `workflow.EXPTDIR`, but the expressions with cycle-specific references remain.
<!--cell 12-->

In [7]:
experiment_config.dereference()
print(experiment_config)

task_make_ics:
  chgres_cube:
    execution:
      executable: execdir/chgres_cube
    namelist:
      update_values:
        config:
          cycle_day: !int '{{ cycle.strftime(''%d'') }}'
          cycle_hour: !int '{{ cycle.strftime(''%H'') }}'
          cycle_mon: !int '{{ cycle.strftime(''%m'') }}'
          convert_atm: true
          convert_nst: true
          convert_sfc: true
          data_dir_input_grid: /path/to/my/output/make_ics
          external_model: GFS
          input_type: gaussian_nemsio
          mosaic_file_target_grid: path/to/example_mosaic.halo.nc
          tg3_from_soil: false
          tracers:
          - sphum
          - liq_wat
          tracers_input:
          - spfh
          - clwmr
          varmap_file: /path/to/ufs-srweather-app/parm/ufs_utils/varmap_tables/GSDphys_var_map.txt
          vcoord_file_target_grid: path/to/global_hyblev.165.txt
    rundir: /path/to/my/output/make_ics
user:
  ACCOUNT: zrtrr
  MACHINE: hera
  PARMdir: /path/to/ufs-sr

To catch potential errors in the `chgres_cube:` block of the config, the `uwtools` `chgres_cube` driver is used here to validate the block using the chgres-cube schema. The driver requires a `cycle` parameter with a datetime specified, so the current time is used here.
<!--cell 14-->

In [8]:
driver = chgres_cube.ChgresCube(
    config=experiment_config['task_make_ics'],
    cycle=datetime.now()
)
driver.validate()

[2024-11-18T13:34:51]     INFO Validating config against internal schema: chgres-cube
[2024-11-18T13:34:51]     INFO 0 UW schema-validation errors found
[2024-11-18T13:34:51]     INFO Validating config against internal schema: platform
[2024-11-18T13:34:51]     INFO 0 UW schema-validation errors found
[2024-11-18T13:34:51]     INFO 20241118 13Z chgres_cube valid schema: State: Ready


Asset(ref=None, ready=<function Assets.validate.<locals>.<lambda> at 0x7c3664345940>)